# Machine Learning

## 1. Initial Setup

- Set variables and hyperparameters
- Import libraries
- Load dataset

In [1]:
# Max allowable epochs for each model type
MAX_EPOCHS = 2000

# Hyperparameters for Hyperparameter Optimization using Optuna
OPTUNA_N_JOBS = 8                   # Number of parallel jobs for hyperparameter optimization - controls how many trials run simultaneously
OPTUNA_N_TRIALS = 40                # Total number of optimization trials to run - more trials generally lead to better hyperparameter discovery
OPTUNA_PRUNE_N_STARTUP_TRIALS = 8   # Number of random trials before pruning starts - ensures diverse exploration before early stopping
OPTUNA_PRUNE_WARMUP_STEPS = 8       # Number of steps to wait before pruning can occur - prevents premature termination of promising trials


# Temp hyperparameters
# TODO: Remove these temporary hyperparameters once the final configuration is established
MAX_EPOCHS = 10
OPTUNA_N_JOBS = 2                   # Number of parallel jobs for hyperparameter optimization - controls how many trials run simultaneously
OPTUNA_N_TRIALS = 2                # Total number of optimization trials to run - more trials generally lead to better hyperparameter discovery
OPTUNA_PRUNE_N_STARTUP_TRIALS = 0   # Number of random trials before pruning starts - ensures diverse exploration before early stopping
OPTUNA_PRUNE_WARMUP_STEPS = 0       # Number of steps to wait before pruning can occur - prevents premature termination of promising trials


In [2]:
PROCESSED_PATH = "../data/ANA HIDROWEB/RIO MEIA PONTE/processed.csv"

In [3]:
# Python native libraries
from typing import List, Dict, Any, Tuple
import os
import warnings
import uuid

# Data manipulation and preprocessing
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# PyTorch libraries and derivatives
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import lightning as L
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger
from torchinfo import summary

# Optuna Libraries
import optuna
from optuna.integration import PyTorchLightningPruningCallback

# Set precision for matrix multiplication in order to optimize performance
torch.set_float32_matmul_precision("high")

# Set random seed for reproducibility
L.seed_everything(42)

/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
Seed set to 42


42

In [4]:
df = pd.read_csv(
    PROCESSED_PATH,
    sep=";",
    parse_dates=["date"],
    dayfirst=True,
)

df.set_index("date", inplace=True)
df.index = pd.to_datetime(df.index)

df.head()

,rain_upstream_mean,rain_upstream_max,rain_upstream_min,rain_upstream_q25,rain_upstream_q75,level_upstream_mean,level_upstream_max,level_upstream_min,level_upstream_q25,level_upstream_q75,...,flow_after_q75,rain_upstream_acc_2_days,rain_downstream_acc_2_days,rain_after_acc_2_days,rain_upstream_acc_3_days,rain_downstream_acc_3_days,rain_after_acc_3_days,date_sin,date_cos,year
date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,0.000000,0.0,0.0,0.0,0.0,219.083333,222.0,216.0,217.0,221.00,...,69.60,0.000000,0.008696,0.0,0.000000,0.026087,0.0,0.000000,1.000000,2014
2014-01-02,0.008696,0.2,0.0,0.0,0.0,222.956522,226.0,219.0,221.5,224.00,...,78.10,0.008696,0.217391,0.0,0.008696,0.226087,0.0,0.017213,0.999852,2014
2014-01-03,0.000000,0.0,0.0,0.0,0.0,225.318182,231.0,219.0,220.5,230.00,...,76.30,0.008696,0.217391,0.0,0.008696,0.217391,0.0,0.034422,0.999407,2014
2014-01-04,0.000000,0.0,0.0,0.0,0.0,213.863636,221.0,208.0,212.0,215.75,...,62.85,0.000000,0.000000,0.0,0.008696,0.217391,0.0,0.051620,0.998667,2014
2014-01-05,0.000000,0.0,0.0,0.0,0.0,204.904762,208.0,201.0,204.0,206.00,...,57.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.068802,0.997630,2014


## 2. Data Preprocessing

- Conferir ausência de dados nulos
- Retirar as colunas de vazão, uma vez que está altamente correlacionada à coluna de nível
- Normalização dos dados

In [5]:
# Count missing values in each column
def print_missing_values(data):
    missing_values = data.isnull().sum()
    print("Missing values in each column:")
    print(missing_values[missing_values > 0])

print_missing_values(df)

Missing values in each column:
Series([], dtype: int64)


In [6]:
# Remove columns containing 'flow' from the dataframe
df = df.loc[:, ~df.columns.str.contains('flow')]
df.head()

,rain_upstream_mean,rain_upstream_max,rain_upstream_min,rain_upstream_q25,rain_upstream_q75,level_upstream_mean,level_upstream_max,level_upstream_min,level_upstream_q25,level_upstream_q75,...,level_after_q75,rain_upstream_acc_2_days,rain_downstream_acc_2_days,rain_after_acc_2_days,rain_upstream_acc_3_days,rain_downstream_acc_3_days,rain_after_acc_3_days,date_sin,date_cos,year
date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,0.000000,0.0,0.0,0.0,0.0,219.083333,222.0,216.0,217.0,221.00,...,291.0,0.000000,0.008696,0.0,0.000000,0.026087,0.0,0.000000,1.000000,2014
2014-01-02,0.008696,0.2,0.0,0.0,0.0,222.956522,226.0,219.0,221.5,224.00,...,309.0,0.008696,0.217391,0.0,0.008696,0.226087,0.0,0.017213,0.999852,2014
2014-01-03,0.000000,0.0,0.0,0.0,0.0,225.318182,231.0,219.0,220.5,230.00,...,302.0,0.008696,0.217391,0.0,0.008696,0.217391,0.0,0.034422,0.999407,2014
2014-01-04,0.000000,0.0,0.0,0.0,0.0,213.863636,221.0,208.0,212.0,215.75,...,279.0,0.000000,0.000000,0.0,0.008696,0.217391,0.0,0.051620,0.998667,2014
2014-01-05,0.000000,0.0,0.0,0.0,0.0,204.904762,208.0,201.0,204.0,206.00,...,270.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.068802,0.997630,2014


In [7]:
scaler = StandardScaler()
# Normalize all columns except 'date_sin' and 'date_cos' (if present)
cols_to_normalize = [col for col in df.columns if col not in ["date_sin", "date_cos"]]
df_normalized = df.copy()
df_normalized[cols_to_normalize] = scaler.fit_transform(df[cols_to_normalize])
df_normalized.head()

,rain_upstream_mean,rain_upstream_max,rain_upstream_min,rain_upstream_q25,rain_upstream_q75,level_upstream_mean,level_upstream_max,level_upstream_min,level_upstream_q25,level_upstream_q75,...,level_after_q75,rain_upstream_acc_2_days,rain_downstream_acc_2_days,rain_after_acc_2_days,rain_upstream_acc_3_days,rain_downstream_acc_3_days,rain_after_acc_3_days,date_sin,date_cos,year
date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,-0.28545,-0.287747,0.0,-0.022316,-0.115149,0.245701,0.105763,0.354992,0.294064,0.201384,...,0.663710,-0.371786,-0.341985,-0.334039,-0.428193,-0.344398,-0.383211,0.000000,1.000000,-1.581277
2014-01-02,-0.20383,-0.200634,0.0,-0.022316,-0.115149,0.301341,0.157947,0.400498,0.360028,0.243211,...,0.829041,-0.318631,0.690675,-0.334039,-0.387379,0.416068,-0.383211,0.017213,0.999852,-1.581277
2014-01-03,-0.28545,-0.287747,0.0,-0.022316,-0.115149,0.335268,0.223177,0.400498,0.345369,0.326865,...,0.764746,-0.318631,0.690675,-0.334039,-0.387379,0.383004,-0.383211,0.034422,0.999407,-1.581277
2014-01-04,-0.28545,-0.287747,0.0,-0.022316,-0.115149,0.170716,0.092717,0.233644,0.220771,0.128186,...,0.553489,-0.371786,-0.385012,-0.334039,-0.387379,0.383004,-0.383211,0.051620,0.998667,-1.581277
2014-01-05,-0.28545,-0.287747,0.0,-0.022316,-0.115149,0.042016,-0.076881,0.127465,0.103503,-0.007752,...,0.470823,-0.371786,-0.385012,-0.334039,-0.428193,-0.443589,-0.383211,0.068802,0.997630,-1.581277


## 3. Criação dos Datasets

- Criar dataset para série temporal
- Dividir o dataset em treino, validação e teste

**A divisão dos dados**

A divisão dos dados neste projeto de séries temporais é feita em dois estágios principais:

1. **Chunking aleatório:**  
   Todo o conjunto de dados é dividido em vários "chunks" (blocos) de tamanho aleatório, variando entre 24 e 100 amostras cada. O último chunk é descartado caso tenha menos de 20 amostras, garantindo que cada bloco tenha tamanho suficiente para conter padrões temporais relevantes.

2. **Split sequencial dentro de cada chunk:**  
   Cada chunk é dividido sequencialmente em três partes: treino (60%), validação (20%) e teste (20%). Assim, cada bloco preserva a ordem temporal dos dados, e cada divisão é feita respeitando a sequência original.

---

**Por que usar chunking em séries temporais?**

- Ao dividir em chunks aleatórios, aumentamos a diversidade dos padrões apresentados ao modelo, pois cada chunk pode capturar diferentes regimes, sazonalidades ou comportamentos do sistema ao longo do tempo.
- Chunking reduz o risco de o modelo se especializar apenas em um período específico da série, promovendo maior generalização.
- Essa abordagem é especialmente útil quando há mudanças de regime, rupturas ou eventos raros distribuídos ao longo do tempo, pois aumenta a chance de tais eventos aparecerem em diferentes splits (treino, validação, teste).

---

**Comparação com divisão sequencial tradicional**

Na divisão sequencial tradicional (por exemplo, 60% inicial para treino, 20% seguinte para validação e 20% final para teste), temos:

- O modelo treina apenas com os dados do início da série, valida com dados intermediários e testa apenas com o final da série.
- Isso pode ser problemático se houver tendências, sazonalidades ou mudanças de regime ao longo do tempo, pois o modelo pode não ser exposto a todos os padrões relevantes durante o treinamento.
- Eventos raros ou extremos podem ficar concentrados em apenas um dos splits, prejudicando a avaliação do modelo.

---

**Benefícios do chunking aleatório para séries temporais**

- Diversifica os contextos temporais em cada split, tornando a avaliação mais robusta.
- Ajuda a simular cenários de produção, onde o modelo pode ser aplicado em diferentes períodos e condições.
- Mantém a ordem temporal dentro de cada chunk, respeitando a estrutura de dependência temporal essencial para séries temporais.
- Reduz o risco de overfitting a um único regime temporal.

---

*Em resumo: chunking aleatório seguido de split sequencial dentro de cada chunk é uma estratégia que preserva a ordem temporal local, aumenta a diversidade dos dados em cada etapa do treinamento e avaliação, e proporciona uma avaliação mais realista e robusta do desempenho do modelo em séries temporais.*

In [8]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length=24, target_col="level_downstream_max"):
        self.data = data.reset_index(drop=True)
        self.sequence_length = sequence_length
        self.target_col = self.data.columns.get_loc(target_col)

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        x = self.data.iloc[
            idx : idx + self.sequence_length
        ].values  # (sequence_length, num_features)
        y = self.data.iloc[idx + self.sequence_length, self.target_col]  # scalar value
        return torch.tensor(x, dtype=torch.float32), torch.tensor(
            y, dtype=torch.float32
        )


class NonOverlappingConcatDataset(Dataset):
    def __init__(self, datasets):
        self.datasets = datasets
        self.cumulative_lengths = []
        total = 0
        for d in datasets:
            self.cumulative_lengths.append(total)
            total += len(d)
        self.total_length = total

    def __len__(self):
        return self.total_length

    def __getitem__(self, idx):
        # Find which dataset this idx belongs to
        for i in range(len(self.datasets)):
            if idx < self.cumulative_lengths[i] + len(self.datasets[i]):
                local_idx = idx - self.cumulative_lengths[i]
                return self.datasets[i][local_idx]
        raise IndexError("Index out of range")


def split_train_validation_test(
    data: pd.DataFrame, train_size: float = 0.6, val_size: float = 0.2
):
    train_end = int(len(data) * train_size)
    val_end = int(len(data) * (train_size + val_size))

    train_data = data.iloc[:train_end]
    val_data = data.iloc[train_end:val_end]
    test_data = data.iloc[val_end:]

    return train_data, val_data, test_data


def create_datasets(
    sequence_length: int,
    min_chunk_size: int = 24,
    max_chunk_size: int = 100,
) -> Tuple[
    NonOverlappingConcatDataset,
    NonOverlappingConcatDataset,
    NonOverlappingConcatDataset,
]:
    import random

    datasets_train: List[TimeSeriesDataset] = []
    datasets_validation: List[TimeSeriesDataset] = []
    datasets_test: List[TimeSeriesDataset] = []

    data = df_normalized.reset_index(drop=True)
    n = len(data)
    idx = 0
    chunk_indices = []

    # Generate random chunk indices
    while idx < n:
        chunk_size = random.randint(min_chunk_size, max_chunk_size)
        if idx + chunk_size >= n:
            # Last chunk
            if n - idx < min_chunk_size:
                break  # Discard last chunk if too small
            chunk_indices.append((idx, n))
            break
        else:
            chunk_indices.append((idx, idx + chunk_size))
            idx += chunk_size

    for start, end in chunk_indices:
        chunk = data.iloc[start:end].reset_index(drop=True)
        if not chunk.empty:
            train_data, validation_data, test_data = split_train_validation_test(chunk)
            if (
                len(train_data) > sequence_length
                and len(validation_data) > sequence_length
                and len(test_data) > sequence_length
            ):
                datasets_train.append(
                    TimeSeriesDataset(train_data, sequence_length=sequence_length)
                )
                datasets_validation.append(
                    TimeSeriesDataset(validation_data, sequence_length=sequence_length)
                )
                datasets_test.append(
                    TimeSeriesDataset(test_data, sequence_length=sequence_length)
                )

    train_dataset = NonOverlappingConcatDataset(datasets_train)
    validation_dataset = NonOverlappingConcatDataset(datasets_validation)
    test_dataset = NonOverlappingConcatDataset(datasets_test)

    return train_dataset, validation_dataset, test_dataset


# Debugging example
# fake_train, fake_validation, fake_test = create_datasets(
#     sequence_length=2,
# )  # Example with sequence length of 2

# print(fake_train[0][1])
# fake_train[0][0].shape, fake_train[0][1].shape  # Should be (2, num_features), scalar

In [9]:
# Test for TimeSeriesDataset

# Create a simple DataFrame with increasing integers
test_df = pd.DataFrame({'A': np.arange(10)})

# Window length = 3
ts_dataset = TimeSeriesDataset(test_df, sequence_length=3, target_col='A')

print("Testing TimeSeriesDataset:")
for i in range(len(ts_dataset)):
    x, y = ts_dataset[i]
    print(f"Index {i}: x = {x.squeeze().numpy()}, y = {y.numpy()}")

# Test for NonOverlappingConcatDataset
# Create two small TimeSeriesDatasets
df1 = pd.DataFrame({'A': np.arange(5)})
df2 = pd.DataFrame({'A': np.arange(10, 15)})

ds1 = TimeSeriesDataset(df1, sequence_length=2, target_col='A')
ds2 = TimeSeriesDataset(df2, sequence_length=2, target_col='A')

concat_ds = NonOverlappingConcatDataset([ds1, ds2])

print("\nTesting NonOverlappingConcatDataset")
print("Should not overlap and should concatenate correctly:")
for i in range(len(concat_ds)):
    x, y = concat_ds[i]
    print(f"Index {i}: x = {x.squeeze().numpy()}, y = {y.numpy()}")


Testing TimeSeriesDataset:
Index 0: x = [0. 1. 2.], y = 3.0
Index 1: x = [1. 2. 3.], y = 4.0
Index 2: x = [2. 3. 4.], y = 5.0
Index 3: x = [3. 4. 5.], y = 6.0
Index 4: x = [4. 5. 6.], y = 7.0
Index 5: x = [5. 6. 7.], y = 8.0
Index 6: x = [6. 7. 8.], y = 9.0

Testing NonOverlappingConcatDataset
Should not overlap and should concatenate correctly:
Index 0: x = [0. 1.], y = 2.0
Index 1: x = [1. 2.], y = 3.0
Index 2: x = [2. 3.], y = 4.0
Index 3: x = [10. 11.], y = 12.0
Index 4: x = [11. 12.], y = 13.0
Index 5: x = [12. 13.], y = 14.0


## 4. Definição dos Modelos

In [10]:
class MLP(L.LightningModule):
    def __init__(
        self,
        input_size,
        hidden_layers,
        learning_rate: float = 0.0001,
    ):
        super(MLP, self).__init__()
        self.save_hyperparameters()

        # MLP para cada passo temporal
        layers = []
        in_features = input_size
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(in_features, hidden_size))
            layers.append(nn.ReLU())
            in_features = hidden_size
        self.step_mlp = nn.Sequential(*layers)
        self.final_layer = nn.Linear(in_features, 1)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        # x: (batch, window_length, input_size)
        # Aplica o MLP em cada passo temporal    print("x type:", type(x))
        batch, window, features = x.shape
        x = x.view(-1, features)  # (batch * window_length, input_size)
        out = self.step_mlp(x)  # (batch * window_length, hidden)
        out = out.view(batch, window, -1)  # (batch, window_length, hidden)
        out = out.mean(dim=1)  # (batch, hidden) - agregação temporal
        out = self.final_layer(out)  # (batch, input_size)
        return out.squeeze(-1)  # (batch,)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("test_loss", loss)
        return loss


class LSTM(L.LightningModule):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        learning_rate: float = 0.0001,
    ):
        super(LSTM, self).__init__()
        self.save_hyperparameters()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        out, _ = self.lstm(x)  # (batch, sequence_length, hidden_size)
        return self.fc(out[:, -1, :]).squeeze(-1)  # (batch,)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("test_loss", loss)
        return loss


class GRU(L.LightningModule):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        learning_rate: float = 0.0001,
    ):
        super(GRU, self).__init__()
        self.save_hyperparameters()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :]).squeeze(-1)  # (batch,)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("test_loss", loss)
        return loss


print(summary(MLP(input_size=5, hidden_layers=[10, 10])))
print("\n")
print(summary(LSTM(input_size=5, hidden_size=10, num_layers=2)))
print("\n")
print(summary(GRU(input_size=5, hidden_size=10, num_layers=2)))

Layer (type:depth-idx)                   Param #
MLP                                      --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       60
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       110
│    └─ReLU: 2-4                         --
├─Linear: 1-2                            11
├─MSELoss: 1-3                           --
Total params: 181
Trainable params: 181
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
LSTM                                     --
├─LSTM: 1-1                              1,560
├─Linear: 1-2                            11
├─MSELoss: 1-3                           --
Total params: 1,571
Trainable params: 1,571
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
GRU                                      --
├─GRU: 1-1                               1,170
├─Linear: 1-2                            11
├─MSELoss: 1-3                           --
Total 

## 5. Treinamento dos Modelos

### 5.1. training functions

In [11]:
def train_model(
    model,
    train_data,
    val_data,
    logger,
    epochs,
    batch_size,
    callbacks=[],
    verbose=True,
) -> None:
    """
    Treina o modelo com os dados de treino e validação.
    """

    train_loader = DataLoader(
        train_data, batch_size=batch_size, shuffle=True, drop_last=True
    )
    validation_loader = DataLoader(
        val_data, batch_size=4, shuffle=False, drop_last=True
    )

    trainer = Trainer(
        max_epochs=epochs,
        accelerator="auto",
        devices=1,
        logger=logger,
        enable_progress_bar=verbose,
        log_every_n_steps=1,
        callbacks=callbacks,
        deterministic=True,
        check_val_every_n_epoch=1,
        enable_model_summary=verbose,
    )
    trainer.fit(model, train_loader, validation_loader)


def evaluate_model(model, test_data, logger, batch_size=4):
    """
    Avalia o modelo com os dados de teste.
    """
    print("\nEvaluating model. Using the test dataset.")
    test_loader = DataLoader(
        test_data, batch_size=batch_size, shuffle=False, drop_last=True
    )

    trainer = Trainer(
        accelerator="auto",
        devices=1,
        logger=logger,
        enable_progress_bar=True,
        deterministic=True,
    )
    trainer.test(model, test_loader)


def run_experiment(
    model_class: type[L.LightningModule],
    name: str,
    model_kwargs: Dict[str, Any] = {},
    train_kwargs: Dict[str, Any] = {},
    evaluate: bool = True,
    extra_callbacks: List[L.Callback] = [],
    verbose: bool = True,
    id: str = "",
):
    """
    Executa um experimento de treinamento e avaliação do modelo.
    """
    # Logger
    logger = TensorBoardLogger(save_dir=os.getcwd(), name=f"lightning_logs/{name}")

    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss", mode="min", save_top_k=1, filename=f"best_model_{id}"
    )

    early_stopping_callback = EarlyStopping(
        monitor="val_loss",
        min_delta=1e-4,
        patience=10,
        mode="min",
        strict=False,
    )

    train_dataset, validation_dataset, test_dataset = create_datasets(
        sequence_length=train_kwargs.get("sequence_length", 5)
    )

    # Remove 'sequence_length' from train_kwargs if present
    train_kwargs = dict(train_kwargs)  # make a copy to avoid side effects
    train_kwargs.pop("sequence_length", None)

    input_size = train_dataset[0][0].shape[1]  # Assuming all inputs have the same shape

    model = model_class(input_size=input_size, **(model_kwargs or {}))

    train_model(
        model,
        train_dataset,
        validation_dataset,
        logger=logger,
        **(train_kwargs or {}),
        callbacks=[checkpoint_callback, early_stopping_callback, *extra_callbacks],
        verbose=verbose,
    )

    best_validation_loss = checkpoint_callback.best_model_score
    best_model_path = checkpoint_callback.best_model_path

    # To view the logs, you can use TensorBoard.
    # If 'logs' is a SummaryWriter, get its log_dir and launch TensorBoard:
    print(f"To view logs, run in terminal:\n  tensorboard --logdir {logger.log_dir}")
    print("Best model saved at:", best_model_path)

    if evaluate:
        best_model = model_class.load_from_checkpoint(best_model_path, **model_kwargs)
        evaluate_model(best_model, test_dataset, logger, batch_size=4)

    return {
        "best_validation_loss": (
            best_validation_loss.item() if best_validation_loss is not None else None
        ),
        "best_model_path": best_model_path,
    }

### 5.2. Hyperparameter optimization functions

In [12]:
def get_optimization_args(model_class: L.LightningModule, trial):
    """
    Retorna os argumentos necessários para executar o experimento.
    """
    if model_class == MLP:
        model_kwargs = {
            "hidden_layers": trial.suggest_categorical(
                "hidden_layers",
                (
                    [32, 16],
                    [48, 24],
                    [64, 32],
                    [96, 48],
                    [128, 64],
                    [64, 32, 16],
                    [128, 64, 32],
                ),
            ),
            "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True),
        }
        train_kwargs = {
            "epochs": MAX_EPOCHS,
            "batch_size": trial.suggest_int("batch_size", 2, 32),
            "sequence_length": trial.suggest_int("sequence_length", 2, 15),
        }
    elif model_class == LSTM:
        model_kwargs = {
            "hidden_size": trial.suggest_int("hidden_size", 16, 128),
            "num_layers": trial.suggest_int("num_layers", 1, 3),
            "learning_rate": trial.suggest_float("learning_rate", 1e-7, 1e-3, log=True),
        }
        train_kwargs = {
            "epochs": MAX_EPOCHS,
            "batch_size": trial.suggest_int("batch_size", 2, 32),
            "sequence_length": trial.suggest_int("sequence_length", 2, 15),
        }
    elif model_class == GRU:
        model_kwargs = {
            "hidden_size": trial.suggest_int("hidden_size", 16, 128),
            "num_layers": trial.suggest_int("num_layers", 1, 3),
            "learning_rate": trial.suggest_float("learning_rate", 1e-7, 1e-3, log=True),
        }
        train_kwargs = {
            "epochs": MAX_EPOCHS,
            "batch_size": trial.suggest_int("batch_size", 8, 32),
            "sequence_length": trial.suggest_int("sequence_length", 3, 12),
        }
    else:
        raise ValueError(f"Unsupported model class: {model_class}")
    return model_kwargs, train_kwargs


def get_objective_fn(model_class, name):
    """
    Retorna uma função de objetivo para otimização de hiperparâmetros.
    """

    def objective(trial: optuna.trial.Trial) -> float:
        # Ajusta os hiperparâmetros com base no trial
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning)
            model_kwargs, train_kwargs = get_optimization_args(model_class, trial)

        optuna_pruning_callback = PyTorchLightningPruningCallback(
            trial, monitor="val_loss"
        )

        results = run_experiment(
            model_class=model_class,
            name=name,
            model_kwargs=model_kwargs,
            train_kwargs=train_kwargs,
            evaluate=False,
            extra_callbacks=[optuna_pruning_callback],
            verbose=False,
            id=uuid.uuid4().hex[:8],
        )
        return results["best_validation_loss"]

    return objective


def run_optimization(objective_fn, n_trials=OPTUNA_N_TRIALS) -> optuna.study.Study:
    pruner = optuna.pruners.MedianPruner(
        n_startup_trials=OPTUNA_PRUNE_N_STARTUP_TRIALS,
        n_warmup_steps=OPTUNA_PRUNE_WARMUP_STEPS,
    )
    study = optuna.create_study(direction="minimize", pruner=pruner)
    study.optimize(
        objective_fn, n_trials=n_trials, n_jobs=OPTUNA_N_JOBS, show_progress_bar=True
    )

    print("Best trial:")
    trial = study.best_trial
    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    return study

### 5.3. Training loop

In [13]:
mlp_objective_fn = get_objective_fn(
    MLP,
    "mlp",
)

study_mlp = run_optimization(mlp_objective_fn, n_trials=OPTUNA_N_TRIALS)

mlp_optimized_results = run_experiment(
    MLP,
    "mlp_optimized",
    model_kwargs={
        "hidden_layers": study_mlp.best_trial.params["hidden_layers"],
        "learning_rate": study_mlp.best_trial.params["learning_rate"],
    },
    train_kwargs={
        "epochs": MAX_EPOCHS,
        "batch_size": study_mlp.best_trial.params["batch_size"],
        "sequence_length": study_mlp.best_trial.params["sequence_length"],
    },
    evaluate=True,
    verbose=True,
)

print("\nMLP Optimized Results:", mlp_optimized_results)

[I 2025-06-04 13:03:24,677] A new study created in memory with name: no-name-24a0d2e9-e21c-487d-aed3-1e1e07a7049e


  0%|          | 0/2 [00:00<?, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
GPU available: True (cuda), used: True
HPU available: False, using: 0 HPUs
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
2025-06-04 13:03:25.156083: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-04 13:03:25.157791: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-04 13:03:25.192528: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-0

To view logs, run in terminal:
  tensorboard --logdir /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/mlp/version_22
Best model saved at: /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/mlp/version_22/checkpoints/best_model_06c0bbe4.ckpt
[I 2025-06-04 13:03:33,836] Trial 1 finished with value: 0.3137660026550293 and parameters: {'hidden_layers': [64, 32], 'learning_rate': 0.00020928509088824726, 'batch_size': 29, 'sequence_length': 5}. Best is trial 1 with value: 0.3137660026550293.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type       | Params | Mode 
---------------------------------------------------
0 | step_mlp    | Sequential | 4.6 K  | train
1 | final_layer | Linear     | 33     | train
2 | criterion   | MSELoss    | 0      | train
---------------------------------------------------
4.7 K     Trainable params
0         Non-trainable params
4.7 K     Total params
0.019     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


...


Evaluating model. Using the test dataset.


/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.46941274404525757    │
└───────────────────────────┴───────────────────────────┘


MLP Optimized Results: {'best_validation_loss': 0.45299670100212097, 'best_model_path': '/home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/mlp_optimized/version_1/checkpoints/best_model_.ckpt'}


In [14]:
lstm_objective_fn = get_objective_fn(
    LSTM,
    "lstm",
)
study_lstm = run_optimization(lstm_objective_fn, n_trials=OPTUNA_N_TRIALS)
lstm_optimized_results = run_experiment(
    LSTM,
    "lstm_optimized",
    model_kwargs={
        "hidden_size": study_lstm.best_trial.params["hidden_size"],
        "num_layers": study_lstm.best_trial.params["num_layers"],
        "learning_rate": study_lstm.best_trial.params["learning_rate"],
    },
    train_kwargs={
        "epochs": MAX_EPOCHS,
        "batch_size": study_lstm.best_trial.params["batch_size"],
    },
    evaluate=True,
    verbose=True,
)
print("\nLSTM Optimized Results:", lstm_optimized_results)

[I 2025-06-04 13:03:41,862] A new study created in memory with name: no-name-4b5f88b3-d224-401a-b63a-79342f81ca8b


  0%|          | 0/2 [00:00<?, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
`Trainer.fit` stopped: `max_epochs=10` r

To view logs, run in terminal:
  tensorboard --logdir /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/lstm/version_3
Best model saved at: /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/lstm/version_3/checkpoints/best_model_d1016a40.ckpt
[I 2025-06-04 13:03:49,914] Trial 0 finished with value: 0.5309064388275146 and parameters: {'hidden_size': 25, 'num_layers': 3, 'learning_rate': 0.0001406758385418012, 'batch_size': 31, 'sequence_length': 5}. Best is trial 0 with value: 0.5309064388275146.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params | Mode 
----------------------------------------------
0 | lstm      | LSTM    | 3.6 K  | train
1 | fc        | Linear  | 17     | train
2 | criterion | MSELoss | 0      | train
----------------------------------------------
3.7 K     Trainable params
0         Non-trainable params
3.7 K     Total params
0.015     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


...

To view logs, run in terminal:
  tensorboard --logdir /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/lstm_optimized/version_1
Best model saved at: /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/lstm_optimized/version_1/checkpoints/best_model_.ckpt

Evaluating model. Using the test dataset.


/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.3823595643043518     │
└───────────────────────────┴───────────────────────────┘


LSTM Optimized Results: {'best_validation_loss': 0.3736557960510254, 'best_model_path': '/home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/lstm_optimized/version_1/checkpoints/best_model_.ckpt'}


In [15]:
gru_objective_fn = get_objective_fn(
    GRU,
    "gru",
)
study_gru = run_optimization(gru_objective_fn, n_trials=OPTUNA_N_TRIALS)
gru_optimized_results = run_experiment(
    GRU,
    "gru_optimized",
    model_kwargs={
        "hidden_size": study_gru.best_trial.params["hidden_size"],
        "num_layers": study_gru.best_trial.params["num_layers"],
        "learning_rate": study_gru.best_trial.params["learning_rate"],
    },
    train_kwargs={
        "epochs": MAX_EPOCHS,
        "batch_size": study_gru.best_trial.params["batch_size"],
    },
    evaluate=True,
    verbose=True,
)
print("\nGRU Optimized Results:", gru_optimized_results)

[I 2025-06-04 13:04:08,743] A new study created in memory with name: no-name-3bc0114e-ad03-49ed-b44f-098ccd371c32


  0%|          | 0/2 [00:00<?, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
GPU available: True (cuda), used: True
HPU available: False, using: 0 HPUs
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
`Trainer.fit` stopped: `max_epochs=10` r

To view logs, run in terminal:
  tensorboard --logdir /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/gru/version_1
Best model saved at: /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/gru/version_1/checkpoints/best_model_da3ab101.ckpt
[I 2025-06-04 13:04:13,490] Trial 1 finished with value: 0.6628364324569702 and parameters: {'hidden_size': 79, 'num_layers': 1, 'learning_rate': 8.692968024022598e-07, 'batch_size': 32, 'sequence_length': 10}. Best is trial 1 with value: 0.6628364324569702.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params | Mode 
----------------------------------------------
0 | gru       | GRU     | 16.3 K | train
1 | fc        | Linear  | 37     | train
2 | criterion | MSELoss | 0      | train
----------------------------------------------
16.3 K    Trainable params
0         Non-trainable params
16.3 K    Total params
0.065     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


...

To view logs, run in terminal:
  tensorboard --logdir /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/gru_optimized/version_1
Best model saved at: /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/gru_optimized/version_1/checkpoints/best_model_.ckpt

Evaluating model. Using the test dataset.


/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.40559136867523193    │
└───────────────────────────┴───────────────────────────┘


GRU Optimized Results: {'best_validation_loss': 0.41171616315841675, 'best_model_path': '/home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/gru_optimized/version_1/checkpoints/best_model_.ckpt'}
